In [7]:
import numpy as np
import cv2
import json
from typing import List

In [8]:
class check_img_Exception(Exception):
    def __init__(self, *args: object) -> None:
        super().__init__(*args)
        self.name = args[0]
        
# raise check_img('Wrong!', 'No Reason!')


class Bbox():
    def __init__(self, data: np.ndarray) -> None:
        self.c1 = data[0:2]
        self.c2 = data[2:4]
        self.label = data[4]
        self.Iposi = data[-2:]


class block_image():
    def __init__(self, bbox: List[Bbox]) -> None:
        self.Bboxes = bbox
        if len(self.Bboxes) != len(list(filter(lambda x: x.Iposi == self.Bboxes[0].Iposi, self.Bboxes))):
            raise check_img_Exception('Contain Bbox from other image!')
        self.position = self.Bboxes[0].Iposi
    

In [ ]:
def run_process(block: block_image, up: block_image, down: block_image, left: block_image, right: block_image) -> None:
    # flag = 0,1,2,3对应上右下左
    # 2675 * 1505
    for flag in range(4):
        if 
    

In [26]:
temp_data = np.loadtxt('./results/final_result1.csv', dtype=int, delimiter=',')
h, _ = temp_data.shape
Bboxes1 = [Bbox(temp_data[i]) for i in range(h)]

temp_data = np.loadtxt('./results/final_result2.csv', dtype=int, delimiter=',')
h, _ = temp_data.shape
Bboxes2 = [Bbox(temp_data[i]) for i in range(h)]




# 实现Bbox类，所有Bbox合为Bbox对象的列表

## Pre-Process

In [ ]:
c11 = Bboxes1[:, 0:2]
c12 = Bboxes1[:, 2:4]
label1 = Bboxes1[:, 4]
position1 = Bboxes1[:, -2:]

## PlayGround

In [ ]:
class check_img(Exception):
    def __init__(self, *args: object) -> None:
        super().__init__(*args)
        self.name = args[0]
        
raise check_img('Wrong!')

In [2]:
class Bboxes():
    def __init__(self, path: str) -> None:
        
        with open('./results/config.json', 'r') as file:
            self.config = json.load(file) # 切割数a, b, 原图大小H, W, 切割后大小h, w
        file.close()
        
        temp = np.loadtxt(path, dtype=int, delimiter=',')
        self.c1 = temp[:, 0:2]
        self.c2 = temp[:, 2:4]
        self.label = temp[:, 4]
        self.position = temp[:, -2:-1]
        
    

In [6]:
temp = np.arange(24).reshape(4,6)
list(filter(lambda x: np.sum(x), temp))

[array([0, 1, 2, 3, 4, 5]),
 array([ 6,  7,  8,  9, 10, 11]),
 array([12, 13, 14, 15, 16, 17]),
 array([18, 19, 20, 21, 22, 23])]

In [30]:
from typing import List
vector = List[Bbox]
def test(temp: vector):
    pass